In [1]:
import os
os.chdir("../")
os.getcwd()

'/home/izam/coding/hackathon'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PromptGenerationConfig:
    root_dir: Path
    data_path: Path
    visualization_path: Path
    data_information_file: Path

In [3]:
from gemInsights.constants import *
from gemInsights.utils.common import read_yaml, create_directories

In [5]:
#udpate configuration manager
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        key_filepath = KEY_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.key = read_yaml(key_filepath)

        create_directories([self.config.artifacts_root])


    def get_prompt_generation_config(self) -> PromptGenerationConfig:
        config = self.config.prompt_generation
        
        create_directories([config.root_dir])

        prompt_generation_config = PromptGenerationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            visualization_path=config.visualization_path,
            data_information_file=config.data_information_file,
        )

        return prompt_generation_config

In [10]:
import pandas as pd
from gemInsights.utils.common import load_json
import os
import PIL.Image
from gemInsights import logger

In [16]:
class PromptGeneration:
    def __init__(self, config: PromptGenerationConfig):
        self.config = config

    def generate(self):
        df = pd.read_csv(self.config.data_path)
        info_json = load_json(Path(self.config.data_information_file))
        target_col = info_json.target_col
        additional_info = info_json.additional_info
        df_shape = df.shape
        df_columns = list(df.columns)
        df_describe = str(df.describe())+"\n"+ str(df.describe(include=["O"]))
        main_prompt = "Act as a data analyst. Here is the complete information and visualization images of a dataset. give valuable insights from the data point wise"
        images = []
        image_dir = os.path.join(self.config.visualization_path, target_col)
        image_files = os.listdir(image_dir)
        for image_file in image_files:
            image_path = os.path.join(image_dir, image_file)
            img = PIL.Image.open(image_path)
            images.append(img)

        prompt = [
            main_prompt,
            f"This is the target column of the dataset - '{target_col}'",
            f"Here are some of the informations related to the dataset - '{additional_info}'",
            f"The shape of the dataset is {df_shape}",
            f"The columns in the dataset are {df_columns}",
            f"Here are some of the general statistics related the dataset - {df_describe}",
        ]

        result = prompt +images
        return result

In [17]:
try:
    config = ConfigurationManager()
    prompt_generation_config = config.get_prompt_generation_config()
    prompt_generation = PromptGeneration(config=prompt_generation_config)
    logger.info(prompt_generation.generate())
except Exception as e:
    raise e

[2023-12-20 11:11:54,993: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-20 11:11:54,994: INFO: common: yaml file: gemini_key.json loaded successfully]
[2023-12-20 11:11:54,994: INFO: common: created directory at: artifacts]
[2023-12-20 11:11:54,995: INFO: common: created directory at: artifacts/prompt_generation]
[2023-12-20 11:11:55,002: INFO: common: json file loaded succesfully from: artifacts/data_ingestion/data_info.json]
[2023-12-20 11:11:55,039: INFO: 1341583654: ['Act as a data analyst. Here is the complete information and visualization images of a dataset. give valuable insights from the data point wise', 'This is the target column of the dataset - Rating', 'Here are some of the informations related to the dataset - It is a sales data', 'The shape of the dataset is (1000, 19)', "The columns in the dataset are ['Unnamed: 0.1', 'Unnamed: 0', 'Invoice ID', 'Branch', 'City', 'Customer type', 'Gender', 'Product line', 'Unit price', 'Quantity', 'Tax 5%', 